In [2]:
# This is a new notebook for the NLP stuff. Feel free to rip some of this for the numeric stuff.
##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import string
import re
import unidecode

In [3]:
# Read in the restaurant data
restaurants = pd.read_json('restaurants.json').T

In [4]:
restaurants.shape

(14089, 16)

In [5]:
restaurants.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
le-pain-quotidien-new-york-137,ce605OMECZ3jLZtk6dCDHA,le-pain-quotidien-new-york-137,Le Pain Quotidien,https://s3-media3.fl.yelpcdn.com/bphoto/z-YqDX...,False,https://www.yelp.com/biz/le-pain-quotidien-new...,84,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"{'latitude': 40.7112998613948, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '395 S End Ave', 'address2': '', ...",+16466188930,(646) 618-8930,812.911847
creperie-new-york-6,B8yFj4qt-HgMCbF5S-GkgA,creperie-new-york-6,Creperie,https://s3-media1.fl.yelpcdn.com/bphoto/uydGkc...,False,https://www.yelp.com/biz/creperie-new-york-6?a...,19,"[{'alias': 'creperies', 'title': 'Creperies'}]",2.5,"{'latitude': 40.711547382455, 'longitude': -74...",[],$,"{'address1': '255 Liberty St', 'address2': '',...",,,762.969495
dos-toros-taqueria-new-york-5,WZLhPYaYSFy7M_-Jh1VuNw,dos-toros-taqueria-new-york-5,Dos Toros Taqueria,https://s3-media1.fl.yelpcdn.com/bphoto/fNdfBJ...,False,https://www.yelp.com/biz/dos-toros-taqueria-ne...,132,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 40.712342, 'longitude': -74.01528}",[],$$,"{'address1': '225 Liberty St', 'address2': 'St...",+13472189518,(347) 218-9518,756.603792
umami-burger-new-york-17,tFQRwk9SVOFgpMBZ55UrLw,umami-burger-new-york-17,Umami Burger,https://s3-media2.fl.yelpcdn.com/bphoto/uvquI3...,False,https://www.yelp.com/biz/umami-burger-new-york...,333,"[{'alias': 'burgers', 'title': 'Burgers'}]",3.0,"{'latitude': 40.712344, 'longitude': -74.015511}","[delivery, pickup]",$$,"{'address1': '225 Liberty St', 'address2': 'St...",+19177284400,(917) 728-4400,756.603792
naya-brookfield-place-new-york-3,jhs35REmv8Yo9-Z27KQJSA,naya-brookfield-place-new-york-3,NAYA - Brookfield Place,https://s3-media3.fl.yelpcdn.com/bphoto/Zd6LgX...,False,https://www.yelp.com/biz/naya-brookfield-place...,32,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 40.71156, 'longitude': -74.01533}","[pickup, delivery]",NaN,"{'address1': '225 Liberty St', 'address2': 'Fl...",+19299468404,(929) 946-8404,756.603792


In [6]:
# Read in the review data
# Define dataframe for reviews
reviews = pd.DataFrame()

# Loop through all review .jsons
for letter in string.ascii_lowercase + '0':
    path = "{}/reviews/{}.json".format(os.getcwd(), letter)
    with open(path, "r") as json_file:
        alias_to_reviews = json.loads(json_file.read())

        # For each alias, get reviews
        for alias in alias_to_reviews.keys():
            # Flatten structure into dataframe, track alias for easy groupby/merge with restaurants
            review = pd.json_normalize(alias_to_reviews[alias], sep="_")
            review['alias'] = alias
            
            # Stitch into final dataframe
            reviews = pd.concat([reviews, review], ignore_index=True)
reviews.shape

(331693, 8)

In [7]:
reviews.head()

,photoCount,reviewCount,eliteYear,localizedDate,rating,comment_text,comment_language,alias
0,190.0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,en,ani-ramen-house-new-york
1,580.0,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,en,ani-ramen-house-new-york
2,531.0,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,en,ani-ramen-house-new-york
3,770.0,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,en,ani-ramen-house-new-york
4,692.0,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,en,ani-ramen-house-new-york


In [8]:
# Find restaurants with missing reviews
missing_reviews = restaurants['name'][restaurants['alias'].isin(reviews['alias']) == False].reset_index()
missing_reviews.columns = ['alias', 'name']
missing_reviews

,alias,name
0,sauce-pizzeria-new-york-8,Sauce Pizzeria
1,sams-crispy-chicken-new-york-4,Sam's Crispy Chicken
2,springbone-kitchen-new-york-7,Springbone Kitchen
3,atrio-wine-bar-and-restaurant-new-york-2,Atrio Wine Bar & Restaurant
4,er-hotpot-new-york-4,ER Hotpot
...,...,...
3207,edgewater-sandwich-edgewater,Edgewater Sandwich
3208,india-grill-and-curry-edgewater,India Grill And Curry
3209,plaza-yankees-restaurant-bronx,Plaza Yankees Restaurant
3210,super-m-deli-new-york,Super M Deli


In [9]:
missing_reviews.nunique() # Multiple aliases for restaurants, many to one relationship due to locations

alias    3212
name     3021
dtype: int64

In [10]:
# Paste the dictionary to see if we have missing michelins
mich_dict = {'Eleven Madison Park': 'eleven-madison-park-new-york',
 'Le Bernardin': 'le-bernardin-new-york',
 'Masa': 'masa-new-york',
 'Per Se': 'per-se-new-york',
 'Al Coro': 'al-coro-new-york',
 'Aquavit': 'aquavit-new-york',
 'Aska': 'aska-brooklyn-3',
 'Atera': 'atera-new-york',
 'Atomix': 'atomix-new-york',
 'Blue Hill at Stone Barns': 'family-meal-at-blue-hill-new-york',
 'Daniel': 'daniel-new-york-4',
 'Gabriel Kreuther': 'gabriel-kreuther-new-york',
 'Jean-Georges': 'jean-georges-new-york-2',
 'Jungsik': 'jungsik-new-york',
 'The Modern': 'the-modern-new-york-3',
 'Odo': 'odo-new-york',
 'Saga': 'saga-new-york-6',
 'Sushi Noz': 'sushi-noz-new-york',
 'Sixty Three Clinton': 'sixty-three-clinton-new-york',
 'Bōm': 'bōm-new-york-3',
 'Casa Mono': 'casa-mono-new-york',
 'Caviar Russe': 'caviar-russe-new-york',
 'Clover Hill': 'clover-hill-brooklyn',
 'Cote': 'cote-korean-steakhouse-new-york',
 'Crown Shy': 'crown-shy-new-york-2',
 'Dirt Candy': 'dirt-candy-new-york-2',
 'Essential by Christophe': 'essential-by-christophe-new-york-2',
 'Estela': 'estela-new-york-2',
 'Family Hill at Blue Hill': 'family-meal-at-blue-hill-new-york',
 'The Four Horseman': 'the-four-horsemen-brooklyn',
 'Francie': 'francie-brooklyn',
 'Frevo': 'frevo-new-york',
 'Gramercy Tavern': 'gramercy-tavern-new-york',
 'Hirohisa': 'hirohisa-new-york',
 'Icca': 'icca-new-york',
 'Jeju Noodle Bar': 'jeju-noodle-bar-new-york',
 'Jōji': 'joji-new-york-2',
 'Joomak Banjum': 'joomak-banjum-new-york',
 'Jua': 'jua-new-york',
 'Kochi': 'kochi-new-york',
 'Kosaka': 'kosaka-new-york-2',
 "L'Abeille": 'l-abeille-new-york',
 'Le Coucou': 'le-coucou-new-york',
 'Le Jardinier': 'le-jardinier-new-york',
 'Le Pavillon': 'le-pavillon-new-york',
 'Mari': 'mari-new-york-2',
 'Meju': 'meju-long-island-city',
 'The Musket Room': 'the-musket-room-new-york',
 'Noda': 'noda-new-york',
 'Noz 17': 'noz-17-new-york-4',
 'Oiji Mi': 'oiji-mi-new-york-2',
 'One White Street': 'one-white-street-new-york',
 'Oxalis': 'oxalis-brooklyn',
 'Oxomoco': 'oxomoco-brooklyn-2',
 'Red Paper Clip': 'red-paper-clip-new-york',
 'Restaurant Yuu': 'restaurant-yuu-brooklyn',
 'Rezdôra': 'rezdôra-new-york-4',
 'Semma': 'semma-new-york-2',
 'Shion 69 Leonard Street': 'shion-69-leonard-st-new-york',
 'Shmoné': 'shmoné-new-york',
 'Sushi Amane': 'sushi-amane-new-york',
 'Sushi Ichimura': 'sushi-ichimura-new-york-3',
 'Sushi Nakazawa': 'sushi-nakazawa-new-york',
 'Sushi Yasuda': 'sushi-yasuda-new-york',
 'Tempura Matsui': 'tempura-matsui-new-york',
 'Torien': 'torien-new-york',
 'Torrisi': 'torrisi-new-york-2',
 'Tsukimi': 'tsukimi-new-york',
 'Tuome': 'tuome-new-york-2',
 'Vestry': 'vestry-new-york',
 'Yoshino': 'yoshino-new-york-3'
 }

In [13]:
# Check that aliases are present in the reviews, and in the restaurants
mich_dict = pd.Series(mich_dict)
mich_dict

Eleven Madison Park    eleven-madison-park-new-york
Le Bernardin                  le-bernardin-new-york
Masa                                  masa-new-york
Per Se                              per-se-new-york
Al Coro                            al-coro-new-york
                                   ...             
Torrisi                          torrisi-new-york-2
Tsukimi                            tsukimi-new-york
Tuome                              tuome-new-york-2
Vestry                              vestry-new-york
Yoshino                          yoshino-new-york-3
Length: 71, dtype: object

In [15]:
mich_reviews = mich_dict.isin(reviews['alias'])
len(mich_reviews), sum(mich_reviews) # Seems like we are missing 8 michelins in our reviews

(71, 63)

In [16]:
# Let's see if we can find the Michelins in our restaurant data...
mich_res = mich_dict.isin(restaurants['alias'])
len(mich_res), sum(mich_res) # Seems like the same 8 are missing in our restaurant data, which makes sense

(71, 63)

In [17]:
# Let's find the 14, hard to systemitize, need more thought on this part - we don't have their aliases, so I'm just matching names
find_mich = mich_dict[mich_dict.isin(reviews['alias']) == False & mich_dict.isin(restaurants['alias'])].reset_index()
find_mich # This is good news, we are not missing any Manhattan Michelins. Obv makes sense because we only scraped with Manhattan neighborhood

,index,0
0,Aska,aska-brooklyn-3
1,Clover Hill,clover-hill-brooklyn
2,The Four Horseman,the-four-horsemen-brooklyn
3,Francie,francie-brooklyn
4,Meju,meju-long-island-city
5,Oxalis,oxalis-brooklyn
6,Oxomoco,oxomoco-brooklyn-2
7,Restaurant Yuu,restaurant-yuu-brooklyn


In [18]:
# Let's do a merger with restaurant name and reviews for analysis
# Fancy indexing Jeremy, let me cook
names = restaurants[['alias', 'name', 'categories', 'price', 'coordinates']]
names

,alias,name,categories,price,coordinates
le-pain-quotidien-new-york-137,le-pain-quotidien-new-york-137,Le Pain Quotidien,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",$$,"{'latitude': 40.7112998613948, 'longitude': -7..."
creperie-new-york-6,creperie-new-york-6,Creperie,"[{'alias': 'creperies', 'title': 'Creperies'}]",$,"{'latitude': 40.711547382455, 'longitude': -74..."
dos-toros-taqueria-new-york-5,dos-toros-taqueria-new-york-5,Dos Toros Taqueria,"[{'alias': 'mexican', 'title': 'Mexican'}]",$$,"{'latitude': 40.712342, 'longitude': -74.01528}"
umami-burger-new-york-17,umami-burger-new-york-17,Umami Burger,"[{'alias': 'burgers', 'title': 'Burgers'}]",$$,"{'latitude': 40.712344, 'longitude': -74.015511}"
naya-brookfield-place-new-york-3,naya-brookfield-place-new-york-3,NAYA - Brookfield Place,"[{'alias': 'mediterranean', 'title': 'Mediterr...",NaN,"{'latitude': 40.71156, 'longitude': -74.01533}"
...,...,...,...,...,...
super-m-deli-new-york,super-m-deli-new-york,Super M Deli,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",NaN,"{'latitude': 40.82537, 'longitude': -73.9258}"
happy-wok-bronx,happy-wok-bronx,Happy Wok,"[{'alias': 'chinese', 'title': 'Chinese'}]",NaN,"{'latitude': 40.8253575861454, 'longitude': -7..."
justins-pizza-bronx,justins-pizza-bronx,Justin's Pizza,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",$$,"{'latitude': 40.8269227, 'longitude': -73.9263..."
justin-s-pizza-bronx-2,justin-s-pizza-bronx-2,Justin’s Pizza,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",NaN,"{'latitude': 40.82702, 'longitude': -73.92627}"


In [19]:
# Merge dataframes for analysis
df = pd.merge(reviews, names, on='alias')
df.shape

(331693, 12)

In [20]:
df.head()

,photoCount,reviewCount,eliteYear,localizedDate,rating,comment_text,comment_language,alias,name,categories,price,coordinates
0,190.0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
1,580.0,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
2,531.0,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
3,770.0,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
4,692.0,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,en,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."


In [21]:
# Trim what we will not use for efficiency
df.drop(['comment_language'], axis=1, inplace=True)
df.head()

,photoCount,reviewCount,eliteYear,localizedDate,rating,comment_text,alias,name,categories,price,coordinates
0,190.0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
1,580.0,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
2,531.0,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
3,770.0,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
4,692.0,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,ani-ramen-house-new-york,Ani Ramen House,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."


In [22]:
# Flatten the categories to just reflect the title, then we can check if it includes a specific tag for filtering (using .isin())
df['categories'] = df['categories'].apply(lambda x: [item['title'] for item in x])

# Rename columns for ease
df.columns = ['user_photo_count', 'user_review_count', 'elite_year', 'review_date','rating','review','alias','name','categories','price','location']

df.head()

,user_photo_count,user_review_count,elite_year,review_date,rating,review,alias,name,categories,price,location
0,190.0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
1,580.0,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
2,531.0,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
3,770.0,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."
4,692.0,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156..."


In [23]:
# Start some processing of the text
dup_reviews = df[df['review'].duplicated(keep=False)]
dup_reviews # Hmm, so aliases may reflect the same location?? See the 2-bros-pizza example...

# Doesn't look like it here but if folks copy and paste same reviews for different restaurants, first of all, HA, and second we don't want them anyway

,user_photo_count,user_review_count,elite_year,review_date,rating,review,alias,name,categories,price,location
3636,NaN,482.0,NaN,9/27/2022,5.0,There is a very kind and patient employee by t...,angelika-film-center-and-café-new-york-new-york-3,Angelika Film Center & Café - New York,"[Cinema, Cafes, Bars]",$$,"{'latitude': 40.72583042190133, 'longitude': -..."
4933,35.0,182.0,NaN,4/27/2022,4.0,Bottom line up front: check the store name and...,amarone-scarlatto-new-york,Amarone Scarlatto,[Italian],$$,"{'latitude': 40.76171494384402, 'longitude': -..."
4967,35.0,182.0,NaN,4/27/2022,4.0,Bottom line up front: check the store name and...,amarone-ny-new-york-2,Amarone NY,"[Italian, Seafood]",$$,"{'latitude': 40.7617199, 'longitude': -73.99002}"
5963,7798.0,898.0,2023.0,7/30/2017,3.0,I was in and out because I knew exactly what I...,au-bon-pain-new-york-6,Au Bon Pain,[Sandwiches],$,"{'latitude': 40.756959, 'longitude': -73.990495}"
6051,7798.0,898.0,2023.0,9/16/2018,2.0,I was in and out because I knew exactly what I...,au-bon-pain-new-york-47,Au Bon Pain,"[Soup, Breakfast & Brunch, Sandwiches]",NaN,"{'latitude': 40.7564223760822, 'longitude': -7..."
...,...,...,...,...,...,...,...,...,...,...,...
326715,NaN,5.0,NaN,11/14/2023,1.0,The bar is stinky like that old sour beer smel...,1803-new-york,1803 Nyc,"[Cajun/Creole, Seafood, Bars]",$$,"{'latitude': 40.715498, 'longitude': -74.007178}"
326896,2.0,4.0,NaN,3/30/2018,1.0,"I ordered 4 pizzas for my office, and they cam...",5-boroughs-pizza-new-york-2,5 Boroughs Pizza,[Pizza],$,"{'latitude': 40.72138, 'longitude': -74.00497}"
327153,2800.0,274.0,2023.0,7/24/2022,3.0,"pizza itself was delicious, but the signs were...",2-bros-pizza-new-york-4,2 Bros Pizza,[Pizza],$,"{'latitude': 40.7443525488053, 'longitude': -7..."
328269,2800.0,274.0,2023.0,7/24/2022,3.0,"pizza itself was delicious, but the signs were...",2-bros-pizza-new-york-6,2 Bros Pizza,[Pizza],$,"{'latitude': 40.75681, 'longitude': -73.980238}"


In [24]:
# Drop duplicate reviews
df = df[df['review'].duplicated() == False]
df.shape

(331501, 11)

In [26]:
# Get the michelin tag
df['michelin'] = df['alias'].isin(mich_dict)
df['michelin'].sum() / len(df), df['michelin'].sum() # So .8% is Michelin, we should probably sample for a classifier

(0.00850676166889391, 2820)

In [27]:
df.head()

,user_photo_count,user_review_count,elite_year,review_date,rating,review,alias,name,categories,price,location,michelin
0,190.0,1421.0,NaN,8/6/2023,3.0,Overall the food is pretty good but portions c...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156...",False
1,580.0,148.0,2023.0,6/8/2023,4.0,We&#39;ve been to ani ramen in Montclair New J...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156...",False
2,531.0,186.0,2023.0,5/24/2023,4.0,The real deal Japanese rice. Crispy tofu was t...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156...",False
3,770.0,213.0,2023.0,7/18/2023,3.0,This is located within Brookfield place. You c...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156...",False
4,692.0,185.0,2023.0,11/12/2022,1.0,I have been wanting to try Ani Ramen for the l...,ani-ramen-house-new-york,Ani Ramen House,"[Noodles, Ramen]",NaN,"{'latitude': 40.7124742, 'longitude': -74.0156...",False


In [28]:
# Get list of unique chars to see what we may want to remove
set(' '.join(df['review']))

{'지',
 '集',
 '鱒',
 '堂',
 '자',
 '問',
 'ツ',
 'プ',
 '焗',
 '峰',
 '텐',
 '廳',
 '削',
 '強',
 'ŕ',
 '问',
 'ğ',
 '缺',
 '傅',
 '【',
 '려',
 'ź',
 '飾',
 '嫩',
 '肮',
 'マ',
 '為',
 '따',
 '청',
 '酒',
 '样',
 '卖',
 '화',
 '…',
 '敷',
 '美',
 '构',
 '淋',
 '솥',
 '樓',
 '気',
 '里',
 'も',
 '끔',
 'و',
 '來',
 'ớ',
 '锡',
 '验',
 '左',
 '란',
 '집',
 '짬',
 '响',
 '蜂',
 '滋',
 '淡',
 '司',
 '股',
 '4',
 'ヨ',
 '筋',
 '长',
 'H',
 '達',
 '낌',
 '广',
 '코',
 '累',
 '乎',
 '哩',
 '쪽',
 '续',
 'ă',
 '햄',
 'ï',
 'ỏ',
 'ち',
 '迪',
 'Z',
 '切',
 '깊',
 '此',
 '费',
 '찜',
 '扣',
 '難',
 '쁜',
 '鹅',
 '师',
 '些',
 '出',
 '馆',
 '秘',
 '名',
 '토',
 '表',
 '环',
 '沾',
 '皇',
 '던',
 '州',
 '鸭',
 '皮',
 '」',
 '麩',
 '×',
 '半',
 '员',
 '回',
 '越',
 '鵝',
 '取',
 'バ',
 '袋',
 '购',
 '港',
 '蓉',
 '찮',
 '+',
 '紅',
 '去',
 '특',
 '페',
 '啃',
 '闆',
 '流',
 '彼',
 'ど',
 '는',
 '粒',
 '音',
 '涂',
 '连',
 'Á',
 '涼',
 '给',
 '萝',
 '苦',
 '葱',
 '定',
 'ộ',
 '挺',
 '漢',
 '담',
 '棄',
 '显',
 '径',
 '槟',
 '片',
 '겠',
 '贯',
 '트',
 'ス',
 '穷',
 '杀',
 '圈',
 '爾',
 '았',
 '鑊',
 '均',
 'う',
 '퀄',
 '弹',
 'I',
 '發',
 '布'

In [31]:
# Get the counts, let's see frequencies
char_count = df['review'].str.split('').explode().value_counts().to_frame().reset_index()
char_count.columns = ['char', 'count']

In [32]:
# They do say e is the most popular letter in the English language... my childhood fun fact is true
print(char_count)

     char     count
0          31771368
1       e  17193449
2       t  11893759
3       a  11574489
4       o  10080546
...   ...       ...
2391    蕎         1
2392    病         1
2393    铁         1
2394    玻         1
2395    饅         1

[2396 rows x 2 columns]


In [33]:
# Let's try some slices
char_count[char_count['count'] <= 80] # Just other chars it seems

,char,count
134,有,79
135,い,78
136,の,77
137,吃,77
138,た,72
...,...,...
2391,蕎,1
2392,病,1
2393,铁,1
2394,玻,1


In [34]:
df['review'] # See that &#39;... let's see the n-grams now and we can decide how to clean
# &#39; is the code for an apostrophe

0         Overall the food is pretty good but portions c...
1         We&#39;ve been to ani ramen in Montclair New J...
2         The real deal Japanese rice. Crispy tofu was t...
3         This is located within Brookfield place. You c...
4         I have been wanting to try Ani Ramen for the l...
                                ...                        
331688    I honestly think between 161-170th in the area...
331689    Awesome pizza for the area.  Prices can be a l...
331690    I called to order. When I gave my address, whi...
331691    I called this place to place a delivery order....
331692    Nonna&#39;s is quality pizza, and not just for...
Name: review, Length: 331501, dtype: object

In [35]:
from nltk import ngrams, FreqDist
from nltk.tokenize import TreebankWordTokenizer

In [36]:
# Tokenize the review data
tokens = df['review'].apply(TreebankWordTokenizer().tokenize)

# Generate n-grams for each row (try 2 words together)
n_grams = tokens.apply(lambda x: list(ngrams(x, 2)))

In [37]:
n_grams # See that sus index 1

0         [(Overall, the), (the, food), (food, is), (is,...
1         [(We, &), (&, #), (#, 39), (39, ;), (;, ve), (...
2         [(The, real), (real, deal), (deal, Japanese), ...
3         [(This, is), (is, located), (located, within),...
4         [(I, have), (have, been), (been, wanting), (wa...
                                ...                        
331688    [(I, honestly), (honestly, think), (think, bet...
331689    [(Awesome, pizza), (pizza, for), (for, the), (...
331690    [(I, called), (called, to), (to, order.), (ord...
331691    [(I, called), (called, this), (this, place), (...
331692    [(Nonna, &), (&, #), (#, 39), (39, ;), (;, s),...
Name: review, Length: 331501, dtype: object

In [38]:
# Flatten the list of n-grams
all_ngrams = [ngram for row in n_grams for ngram in row]

# Feed into NLTK Freq Dist
freq_dist = FreqDist(all_ngrams)

In [39]:
# Display the top N n-grams
print(freq_dist.most_common(10)) # We'll use this down the line when we start subsetting the data
# HTML patterns are still apparent, let's figure out how to parse this...
# Should've just looked at the JSON and saw the <br> LOL

[(('<', 'br'), 814781), (('br', '>'), 814781), (('&', '#'), 681411), (('#', '39'), 601819), (('39', ';'), 601818), (('>', '<'), 336433), ((';', 't'), 208469), ((';', 's'), 185562), ((',', 'and'), 140263), (('I', '&'), 119291)]


In [40]:
# Remove HTML tags from text using Beautiful Soup
# Why a library over regular expressions? Better edge case handling
from bs4 import BeautifulSoup
df['review'] = df['review'].apply(lambda x: unidecode.unidecode(BeautifulSoup(x, 'html.parser').get_text(separator=' '))) # The warning just means maybe not all contain HTML tags, it's fine. Unidecode to remove non-English chars

/var/folders/_q/gmdf_7c54956tq7fvnbprqw00000gn/T/ipykernel_32247/3999042358.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['review'] = df['review'].apply(lambda x: unidecode.unidecode(BeautifulSoup(x, 'html.parser').get_text(separator=' '))) # The warning just means maybe not all contain HTML tags, it's fine. Unidecode to remove non-English chars


In [41]:
df['review']

0         Overall the food is pretty good but portions c...
1         We've been to ani ramen in Montclair New Jerse...
2         The real deal Japanese rice. Crispy tofu was t...
3         This is located within Brookfield place. You c...
4         I have been wanting to try Ani Ramen for the l...
                                ...                        
331688    I honestly think between 161-170th in the area...
331689    Awesome pizza for the area.  Prices can be a l...
331690    I called to order. When I gave my address, whi...
331691    I called this place to place a delivery order....
331692    Nonna's is quality pizza, and not just for da ...
Name: review, Length: 331501, dtype: object

In [42]:
# Get the counts, let's see frequencies
char_count = df['review'].str.split('').explode().value_counts().to_frame().reset_index()
char_count.columns = ['char', 'count']
char_count

,char,count
0,,32505228
1,e,17192174
2,t,11887240
3,a,11535101
4,o,10078141
...,...,...
91,\r,95
92,},87
93,{,82
94,`,25


In [43]:
# Get the slice of the less frequent, let's inspect
char_count.iloc[65:]

,char,count
65,U,45309
66,4,45284
67,J,38386
68,K,37575
69,&,32821
70,6,23887
71,9,23570
72,8,22471
73,7,21492
74,;,17787


In [44]:
# Remove some of these, rest seem plausible
df['review'] = df['review'].apply(lambda x: re.sub(r'[{}*\r\\\^`|]', '', x))
df['review']

0         Overall the food is pretty good but portions c...
1         We've been to ani ramen in Montclair New Jerse...
2         The real deal Japanese rice. Crispy tofu was t...
3         This is located within Brookfield place. You c...
4         I have been wanting to try Ani Ramen for the l...
                                ...                        
331688    I honestly think between 161-170th in the area...
331689    Awesome pizza for the area.  Prices can be a l...
331690    I called to order. When I gave my address, whi...
331691    I called this place to place a delivery order....
331692    Nonna's is quality pizza, and not just for da ...
Name: review, Length: 331501, dtype: object

In [45]:
set(' '.join(df['review'])) # Now looks cleaner, ready for VADER

{' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~'}

In [46]:
# Now start the analysis